## Prerequisites

Pada bagian kita akan mencoba meng-implementasikan proses training object detection menggunakan tensorflow. Tutorial ini mengangkat studi kasus yaitu mendeteksi muka/wajah. sebagai persiapan awal sebelum meng-implentasikan proses training. Kita diharuskan menyiapkan dan menginstall dependencies yang dibutuhkan pada komputer kita. Tahapn persiapan tersebut dapat dilihat di link https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md  

#####  - Tensorflow-script-model
    - git clone https://github.com/tensorflow/models.git
    - extract models
    - buat folder dengan nama 'workshop_dsw'
    - copy dan paste folder 'models' ke dalam folder 'tensorflow_dsw'
        
##### - Akses sebagai global variabel
- Linux
        
        ```
        #command-line :
        export PYTHONPATH="/home/linux/workshop_dsw/models/research:/home/nodeflux/models/research/slim
        # atau bashrc :
        export PYTHONPATH="/home/linux/workshop_dsw/models/research:/home/nodeflux/models/research/slim:$PYTHONPATH
        ```        
- Windows 
        Go to System -> Advanced system settings -> Environment Variables -> New, and add a variable with the name PYTHON_PATH and these values.   
        - Ubah 'PYTHON_PATH' menjadi PYTHONPATH
        - Tambahkan %PYTHONPATH% di system varibles > PATH dan di user variables > PATH
    ![](path_win.png)

##### -Tensorflow

```
         # CPU: 
         pip install tensorflow         
         # GPU: 
         pip install tensorflow-gpu
         # jika menggunakan anaconda
         conda install tensorflow==1.5.0

```
##### - cocoAPI

```
       pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
```
##### - Protobuf
- Linux
    
    ```    
        $ sudo apt-get install protobuf-compiler

        # From workshop_dsw/models/research/        
        $ cd workshop_dsw/models/research
        $ protoc object_detection/protos/*.proto --python_out=.4
        
     ```
        
- Windows    
    - Download [Windows v3.4.0 release “protoc-3.4.0-win32.zip”](https://github.com/google/protobuf/releases/download/v3.4.0/protoc-3.4.0-win32.zip) 
    - Extract the protoc-3.4.0-win32.zip di program files
    - cd path\to\models\research
    - Execute the protobuf compile
            ```
                “C:\Program Files\protoc-3.4.0-win32\bin\protoc.exe” 
                object_detection/protos/*.proto --python_out=.

             ```
        
##### - Pretrained models
- checkpoint models [ssd_mobilenet_v1_coco](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)
- label map [label_ssd_mobilenet_v1_coco](    https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt)

--------------------------------------
## OUTLINE

#### Setelah memastikan semua dependencies ter-install di lokal komputer, selanjutnya kita akan melakukan beberapa proses yaitu :
1. Data preparation        
2. Genearate Label_map(*.pbtx)                
3. Genearate TFRecord (*.record)
    - train.record
    - valid.record
4. File Config(.*config)
5. Training
6. Export model untuk inferencing

----------------------------------------
## Tahap 1: Data Preparation

Pada bagian ini kita akan menyiapkan dataset untuk melatih model dalam mendeteksi muka. Data yang kita gunakan adalah dataset yang sudah diberi-label sebelumnya. 

Step yang dilakukan :
1. File yang dibutuhkan antara lain :
   - Download Wider dataset 'http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/'  
   
   ![](wider_face.jpg)
   
     File yang dibutuhan :
         a. Wider Face Training Images
         b. Wider Face Validation Images
         c. Face annotations 
   - (optional) labels tools: (https://github.com/tzutalin/labelImg)
   ![](label_tools.jpg)
2. Extract file ke dalam folder yang sama:
    contoh struktur folder:
    
         data------
             -- WIDER_val
             -- WIDER_train
             -- wider_face_split
          
          *) Note : nama folder harus sama, Jika beda, maka pada script 'wider_to_xml_pascal.py' harus ikut diganti.
3. Jalan script 'wider_to_xml_pascal.py' untuk mengkonversi format WIDER dataset ke dalam bentuk format pascal (.xml)

##### Jalankan di command-line :

 ```bash

       # arahkan path ke project direktori
      cd C:\Users\rizky\Documents\dsw\dsw_training
      python wider_to_xml_pascal.py
 ```

---------------------------------
## Tahap 2 : Generate Label Map
Label Map meruapakan sebuah file seperti format .json. File ini dibutuhkan tensorflow sebagai metadata dari dataset. 
contoh format :
```
    item{
        id: 1
        name: 'face' 
     }

    item{
        id: 2
        name: 'body' 
     }
```
Pada step ini, kita akan menggunakan script 'generate_labelmap_xml_pascal.py' untuk meng-generate sebuah file bertipe *.pbtxt dari format pascal.

###### Jalankan di command-line :
- Linux
```
    # -l = file output
    # -p = file xml format pascal
    python generate_label_map_from_xml_pascal.py -p data/tf_wider_train/annotations/xmls -l wider_label_map.pbtxt
```
- Windows
```
   python generate_labelmap_xml_pascal.py -p data\tf_wider_train\annotations\xmls -l wider_label_map.pbtxt
```

----------------------------------------
## Tahap 3: Generate TF Record

TFRecord merupakan format untuk tensorflow yang sudah di enkapsulasi ke dalam bentuk tensor object. Isi tensor object tersebut equivalent dengan list dari dict untuk format python. Untuk setiap dict sendiri berisi list seperti: 
- list dari bbox (xmin,xmax,ymin,ymax) dalam bentuk rasio, 
- list dari nama/ class dari bbox
- path 
- height dari image 
- widht dari image
- dll.

#### Contoh konversi :

    ```
    images_list = [{'height': 400,'width':400,
                    'objects': [{'xmin': 12, 
                                'xmax': 24, 
                                'ymin': 230, 
                                'ymax': 256}, 
                    {'another bbox'},...]
                   ]
                   
    writer = tf.python_io.TFRecordWriter('data/valid.tfrecord')
    
    for i, image in enumerate(images_list):
        height = image['height']  # an int
        width = image['width']  # an int
        
        bbox_Xmins = [float(bbox['xmin']/width) for bbox in image['objects']]  # list of floats
        bbox_Ymins = [float(bbox['ymin']/height) for bbox in image['objects']]  # list of floats
        
        tf_example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(height),
            'image/object/bbox/xmin': dataset_util.float_list_feature(bbox_Xmins)
            'image/object/bbox/ymin': dataset_util.float_list_feature(bbox_Ymins)
            
        }))
        writer.write(tf_example.SerializeToString())
    writer.close()

    ```


##### Jalankan di command-line :

- Linux :

```
# untuk data validasi 
 python generate_tfrecord_from_xml_pascal.py \
--image_dir=/home/nodeflux/workshop/dsw/dsw_training/data/tf_wider_val/images \
--label_map_path=/home/nodeflux/workshop/dsw/dsw_training/wider_label_map.pbtxt \
--output_path=/home/nodeflux/workshop/dsw/dsw_training/wider_val.record \
--xml_dir=/home/nodeflux/workshop/dsw/dsw_training/data/tf_wider_val/annotations/xmls

# untuk data train
 python generate_tfrecord_from_xml_pascal.py \
--image_dir=/home/nodeflux/workshop/dsw/dsw_training/data/tf_wider_train/images \
--label_map_path=/home/nodeflux/workshop/dsw/dsw_training/wider_label_map.pbtxt \
--output_path=/home/nodeflux/workshop/dsw/dsw_training/tf_wider_train.record \
--xml_dir=/home/nodeflux/workshop/dsw/dsw_training/data/tf_wider_train/annotations/xmls
```

- Windows
     
```
     # untuk data validasi 
     python generate_tfrecord_from_xml_pascal.py --image_dir=C:\Users\rizky\Documents\dsw\dsw_training\data\tf_wider_val\images --label_map_path=C:\Users\rizky\Documents\dsw\dsw_training\wider_label_map.pbtxt --output_path=C:\Users\rizky\Documents\dsw\dsw_training\wider_val.record --xml_dir=C:\Users\rizky\Documents\dsw\dsw_training\data\tf_wider_val\annotations\xmls

    # untuk data train
    python generate_tfrecord_from_xml_pascal.py --image_dir=C:\Users\rizky\Documents\dsw\dsw_training\data\tf_wider_train\images --label_map_path=C:\Users\rizky\Documents\dsw\dsw_training\wider_label_map.pbtxt --output_path=C:\Users\rizky\Documents\dsw\dsw_training\wider_train.record --xml_dir=C:\Users\rizky\Documents\dsw\dsw_training\data\tf_wider_train\annotations\xmls

    ```

------------------------------------------------
## Tahap 4 :  File Config

Pada tahap ini, kita akan melakukan configurasi arsitektur untuk mendeteksi wajah. Pada kasus ini, kita gunakan ssd_mobilenet_v1_coco.config sebagai arsitektur model. 

#### Step pada tahap ini antara lain :


1. Download ssd_mobilenet_v1_coco.config
   Kita dapat mendwonload https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs atau mencari di direktori ```models/research/object_detection/samples/configs```, Kemudian copy dan paste file tersebut ke dalam folder project kita agar lebih mudah di akses. 
   
2. Buka ssd_mobilenet_v1_coco.config menggunakan tools text editor. selanjutnya kita ubah beberapa line sesuai kebutuhan kita.
    Adapun bagian-bagian yang diubah yaitu :

    1. num_classes : 'num_classes: 90' menjadi 'num_classes: 1'
    2. fine_tune_checkpoint : Jika kita menggunakan arsitektur yang sudah ada, baiknya kita menggunakan model sebelumnya sebagai fine_tune(transfer knowladge) untuk model kita yang baru.
        ```
        fine_tune_checkpoint: "ssd_mobilenet_v1_coco_11_06_2017/model.ckpt"
        ```

    3. jumlah iterasi (num_steps): num_steps: 500 (bebas)

    4. train_input_reader (path)

        ```
        train_input_reader: {
          tf_record_input_reader {
            input_path: "wider_train.record"
          }
          label_map_path: "wider_label_map.pbtxt"
        }

        ```

    5. eval_input_reader (path)

        ```
        eval_input_reader: {
          tf_record_input_reader {
            input_path: "wider_val.record"
          }
          label_map_path: "wider_label_map.pbtxt"
          shuffle: false
          num_readers: 1
        }


        ```


3.Simpan (save as).config baru

##### Tips :

In the data augmentation section of the training pipeline, some options can be added or removed to try and make the training better. Some of the options are listed here :
- NormalizeImage normalize_image = 1;
- RandomHorizontalFlip random_horizontal_flip = 2;
- RandomPixelValueScale random_pixel_value_scale = 3;
- RandomImageScale random_image_scale = 4;
- RandomRGBtoGray random_rgb_to_gray = 5;
- RandomAdjustBrightness random_adjust_brightness = 6;
- RandomAdjustContrast random_adjust_contrast = 7;
- RandomAdjustHue random_adjust_hue = 8;
- RandomAdjustSaturation random_adjust_saturation = 9;
- RandomDistortColor random_distort_color = 10;
- RandomJitterBoxes random_jitter_boxes = 11;
- RandomCropImage random_crop_image = 12;
- RandomPadImage random_pad_image = 13;
- RandomCropPadImage random_crop_pad_image = 14;
- RandomCropToAspectRatio random_crop_to_aspect_ratio = 15;
- RandomBlackPatches random_black_patches = 16;
- RandomResizeMethod random_resize_method = 17;
- ScaleBoxesToPixelCoordinates scale_boxes_to_pixel_coordinates = 18;
- ResizeImage resize_image = 19;
- SubtractChannelMean subtract_channel_mean = 20;
- SSDRandomCrop ssd_random_crop = 21;
- SSDRandomCropPad ssd_random_crop_pad = 22;
- SSDRandomCropFixedAspectRatio ssd_random_crop_fixed_aspect_ratio = 23;

Example : 

```

  data_augmentation_options {
    ssd_random_crop {
    }
  }
  data_augmentation_options {
    random_pixel_value_scale {
      minval: 0.6
    }
  }
  ```

-----------------------------------------
## Tahap 5 : Training
Pada tahap ini kita sudah dapat melatih object detection menggunakan framework tensorflow menggunanakan WIDER dataset sebagai data train. Kita dapat melatih ini dengan tekhnologi CPU atau GPU tergantung kemampuan komputer kita dan installasi tensorflow.

##### Note: sebelum memulai proses training, buat folder bernama 'training' untuk menyimpan model atau hasil proses training

##### Untuk memulai training Jalankan di command-line :

- Linux 
    ```

    python ~/workshop_dsw/models/research/object_detection/train.py --logtostderr --pipeline_config_path=ssd_mobilenet_v1_face.config  --train_dir=training

    ```

- Windows

    ```
    python C:\Users\rizky\Documents\dsw\models\research\object_detection\train.py --logtostderr --pipeline_config_path=ssd_mobilenet_v1_coco.config  --train_dir=training

    ```
    
    ![Gambar proses training](training_vis.jpg)

Hasil dari model akan terbentuk 3 file checkpoint yang tersimpan di folder training:
   
    1. model.ckpt-${CHECKPOINT_NUMBER}.data-00000-of-00001
    2. model.ckpt-${CHECKPOINT_NUMBER}.index
    3. model.ckpt-${CHECKPOINT_NUMBER}.meta
    
   

##### Note : Untuk mem-visualisasikan hasil traning, maka jalan perintahdi command-line:
  
  ```
  $ tensorboard --logdir=training/
  ```


-----------------------------------
## Tahap 6 : Export model untuk inferencing

Setelah melakukan pelatihan, tahap selanjutnya adalah meng-export model ke dalam bentuk graph proto (*.pb) 
dari file :
```
* model.ckpt-${CHECKPOINT_NUMBER}.data-00000-of-00001,
* model.ckpt-${CHECKPOINT_NUMBER}.index
* model.ckpt-${CHECKPOINT_NUMBER}.meta
```



##### Untuk memulai training Jalankan di command-line :


- Linux
    ``` 
    
    python ~/workshop/dsw/models/research/object_detection/export_inference_graph.py \
        --input_type image_tensor \
        --pipeline_config_path ssd_mobilenet_v1_coco.config \
        --trained_checkpoint_prefix training/0/model.ckpt-500 \
        --output_directory training

    ```
- Windows :
 ```
     python C:\Users\rizky\Documents\dsw\models\research\object_detection\export_inference_graph.py --input_type image_tensor --pipeline_config_path ssd_mobilenet_v1_coco.config --trained_checkpoint_prefix training/model.ckpt-500 --output_directory training

    ```

Hasil export berupa sebuah file bernama ```output_inference_graph.pb ```


--------------------------------------------------
## References :
- https://github.com/qdraw/tensorflow-face-object-detector-tutorial
- http://mmlab.ie.cuhk.edu.hk/projects/WIDERFace/
- https://github.com/tensorflow/models/tree/master/research/object_detection
- https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md
- https://medium.com/@rohitrpatil/how-to-use-tensorflow-object-detection-api-on-windows-102ec8097699